In [1]:
%load_ext autoreload
%autoreload 2

# 1. Tiến hành đọc hết tất cả các review của các sản phẩm vào một dataframe duy nhất

In [2]:
import pandas as pd
import modules.utils as Utils

In [3]:
men_reviews = Utils.getFiles("../00. Data Scraping/data/shopee_reviews/men/")
women_reviews = Utils.getFiles("../00. Data Scraping/data/shopee_reviews/women/")

In [4]:
len(men_reviews) + len(women_reviews)

2151

> **Nhận xét**
> * Có tổng cộng 2151 sản phẩm

In [5]:
men_reviews[:5]

['product_71.csv',
 'product_694.csv',
 'product_898.csv',
 'product_1075.csv',
 'product_702.csv']

In [6]:
df = pd.read_csv("../00. Data Scraping/data/shopee_reviews/men/" + men_reviews[0])

In [7]:
for path in men_reviews[1:]:
    tmp_df = pd.read_csv("../00. Data Scraping/data/shopee_reviews/men/" + path)
    df = pd.concat([df, tmp_df], axis=0).reset_index(drop=True)

In [8]:
for path in women_reviews:
    tmp_df = pd.read_csv("../00. Data Scraping/data/shopee_reviews/women/" + path)
    df = pd.concat([df, tmp_df], axis=0).reset_index(drop=True)

In [9]:
df.shape

(376706, 2)

In [11]:
df

,comment,rating
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5
...,...,...
376701,Giao hàng quá chậm. 10 ngày mới có hàng. Sản p...,5
376702,Thời gian giao hàng dài. Chất vải tạm ổn với g...,4
376703,Cũng ổn nhìn hay hay,4
376704,"nhỏ xíu , ship 2 tuần , thất vọng",5


> **Nhận xét**:
> * Có tổng cộng 376,706 review

🚩 Ghi ra file csv

In [13]:
df.to_csv("./data/data.csv")

<hr>

# 2. Tìm hiểu dữ liệu

🚩 Đánh nhãn cho dữ liệu, nếu rating của khách hàng là:
* $\leq 2$: Tiêu cực
* $\in [3, 4]$: Bình thường
* $= 5$: Tích cực

In [1]:
import pandas as pd

In [4]:
def rating(pScore):
    if pScore <= 2:
        return -1
    
    if pScore <= 4:
        return 0
    
    return 1

In [2]:
df = pd.read_csv("./data/data.csv")

In [5]:
df['label'] = df['rating'].apply(lambda score: rating(score))

In [6]:
df

,Unnamed: 0,comment,rating,label
0,0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1
1,1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1
2,2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1
3,3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1
4,4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1
...,...,...,...,...
376701,376701,Giao hàng quá chậm. 10 ngày mới có hàng. Sản p...,5,1
376702,376702,Thời gian giao hàng dài. Chất vải tạm ổn với g...,4,0
376703,376703,Cũng ổn nhìn hay hay,4,0
376704,376704,"nhỏ xíu , ship 2 tuần , thất vọng",5,1


🚩 Đếm nhóm theo `label` và đếm xem có bao nhiêu phần từ thuộc nhóm đó

In [9]:
df.groupby('label').agg('count')

,Unnamed: 0,comment,rating
label,,,
-1,5525,5525,5525
0,20822,20822,20822
1,350359,350359,350359


In [10]:
# a['emoji'] = a['comment'].apply(lambda comment: extract_emoji(comment))
